In [176]:


import numpy as np
from matplotlib import pyplot as plt

from qutrit_rpe import *

%load_ext autoreload 
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [177]:
xvec = random_error_vector(0.001, 0.01)
qids = ['Q0', 'Q1']
depol_rates = [0.001, 0.01] # 1 and 2 qubit depolarizing rates

In [178]:
parse_error_vector(xvec, qids)

{'single_qutrit': {'Q0': {'X01': -0.026485686422796296,
   'phase01': 0.04233818371753959,
   'X12': 0.025244382112121707,
   'phase12': -0.01108740099847887},
  'Q1': {'X01': -0.012701203704893256,
   'phase01': 0.019331365742392224,
   'X12': 0.022335606059599986,
   'phase12': -0.051140980602201694}},
 'two_qutrit': {'phi1': -0.07294154615391833,
  'phi2': -0.19647825481536127,
  'phi3': 0.026841823634807183,
  'phi4': -0.06829027518388735,
  'phi5': 0.08930114994562333,
  'phi6': 0.024752636743452988,
  'phi7': 0.044429863582980574,
  'phi8': -0.010193917403466646}}

In [179]:
target_model = make_two_qutrit_model(np.zeros(16), single_qutrit_depol=0.0, two_qutrit_depol=0.0, qids=qids)
noise_model = make_two_qutrit_model(xvec, single_qutrit_depol=depol_rates[0], two_qutrit_depol=depol_rates[1], qids=qids)

In [185]:
from numpy.linalg import matrix_power

def gX01(qid):
    return [(f'Gx01', qid)]

def gY01(qid):
    return [(f'Gz01', qid)] + [(f'Gx01', qid)] + [(f'Gx01', qid)]*3

def gX12(qid):
    return [(f'Gx12', qid)]

def gY12(qid):
    return [(f'Gz12', qid)] + [(f'Gx12', qid)] + [(f'Gx12', qid)]*3

def gZ01(qid):
    return [(f'Gz01', qid)]

def gZ12(qid):
    return [(f'Gz12', qid)]

def gX01_inv(qid):
    return [(f'Gx01', qid)]*3 + [(f'Gz12', qid)]*2

def gX12_inv(qid):
    return [(f'Gx12', qid)]*3 + [(f'Gz01', qid)]*2

def gY01_inv(qid):
    return gY01(qid)*3 + [(f'Gz12', qid)]*2

def gY12_inv(qid):
    return gY12(qid)*3 + [(f'Gz01', qid)]*2

def check_inverse_defs():
    ux01 = modelX01(0,0)
    uz01 = modelZ01()
    ux12 = modelX12(0,0)
    uz12 = modelZ12()
    uy01 = matrix_power(uz01, 3)@ux01@uz01
    uy12 = matrix_power(uz12, 3)@ux12@uz12

    print(np.all(np.isclose(ux01@(ux01@ux01@ux01@uz12@uz12), -np.eye(3))))
    print(np.all(np.isclose(ux12@(ux12@ux12@ux12@uz01@uz01), -np.eye(3))))
    print(np.all(np.isclose(uy01@(uy01@uy01@uy01@uz12@uz12), -np.eye(3))))
    print(np.all(np.isclose(uy12@(uy12@uy12@uy12@uz01@uz01), -np.eye(3))))
check_inverse_defs()

True
True
True
True


In [186]:
ux01

array([[0.70710678+0.j        , 0.        -0.70710678j,
        0.        +0.j        ],
       [0.        -0.70710678j, 0.70710678+0.j        ,
        0.        +0.j        ],
       [0.        +0.j        , 0.        +0.j        ,
        1.        +0.j        ]])

In [187]:
sigY = np.array([[0, -1j], [1j, 0]])
expm(-1j*np.pi/4*sigY)

array([[ 0.70710678+0.j, -0.70710678+0.j],
       [ 0.70710678+0.j,  0.70710678+0.j]])

In [201]:
ux01 = modelX01(0,0)
uz01 = modelZ01()
ux12 = modelX12(0,0)
uz12 = modelZ12()
uy01 = matrix_power(uz01, 3)@ux01@uz01
uy12 = matrix_power(uz12, 3)@ux12@uz12

print('phase12 cos prep', np.round(uy12@uy01@np.array([1,0,0]), 3))
print('phase12 sin prep', np.round(uy12@ux01@np.array([1,0,0]), 3))

print('phase01 cos prep', np.round(uy01@uy12@ux01@ux01@np.array([1,0,0]), 3))
print('phase01 sin prep', np.round(uy01@ux12@ux01@ux01@np.array([1,0,0]), 3))

phase12 cos prep [0.707-0.j 0.5  -0.j 0.5  +0.j]
phase12 sin prep [0.707+0.j  0.   -0.5j 0.   -0.5j]
phase01 cos prep [0.+0.5j   0.-0.5j   0.-0.707j]
phase01 sin prep [ 0.   +0.5j  0.   -0.5j -0.707+0.j ]


In [140]:
from pygsti.circuits import Circuit

def make_rpe_circuit(germ, prep, meas, depth, line_labels):
    prep_circ = Circuit(prep, line_labels=line_labels)
    germ_circ = Circuit(germ, line_labels=line_labels)
    meas_circ = Circuit(meas, line_labels=line_labels)
    return prep_circ + germ_circ * depth + meas_circ

class RPEDesign1QT:
    def __init__(self, depths, qid, line_labels=None):
        self.depths = depths
        self.qid = qid
        if line_labels is None:
            line_labels = [qid]
        self.line_labels = line_labels
        self.circuit_dict = self._construct()
        self.circ_list = self._make_circuit_list()


    def _make_circuit_list(self):
        circs = []
        for param_label in self.circuit_dict.keys():
            for type_label in self.circuit_dict[param_label].keys():
                circs.extend(self.circuit_dict[param_label][type_label])
        return pygsti.remove_duplicates(circs)

    

    def _construct(self):
        circ_dict = {
            'Phase01': {},
            'Phase12': {},
            'X01 overrot': {},
            'X12 overrot': {}
        }
        circ_dict['X01 overrot']['I'] = self.make_x01_overrot_cos_circuits(self.depths, self.qid, self.line_labels)
        circ_dict['X01 overrot']['Q'] = self.make_x01_overrot_sin_circuits(self.depths, self.qid, self.line_labels)
        circ_dict['X12 overrot']['I'] = self.make_x12_overrot_cos_circuits(self.depths, self.qid, self.line_labels)
        circ_dict['X12 overrot']['Q'] = self.make_x12_overrot_sin_circuits(self.depths, self.qid, self.line_labels)
        circ_dict['Phase01']['I'] = self.make_phase01_cos_circuits(self.depths, self.qid, self.line_labels)
        circ_dict['Phase01']['Q'] = self.make_phase01_sin_circuits(self.depths, self.qid, self.line_labels)
        return circ_dict

    def make_x01_overrot_cos_circuits(self, depths, qid, line_labels):
        return [make_rpe_circuit(gX01(qid), [], [], depth, line_labels) for depth in depths]
    
    def make_x01_overrot_sin_circuits(self, depths, qid, line_labels):
        return [make_rpe_circuit(gX01(qid), gX01(qid), [], depth, line_labels) for depth in depths]
    
    def make_x12_overrot_cos_circuits(self, depths, qid, line_labels):
        return [make_rpe_circuit(gX12(qid), [], [], depth, line_labels) for depth in depths]
    
    def make_x12_overrot_sin_circuits(self, depths, qid, line_labels):
        return [make_rpe_circuit(gX12(qid), gX12(qid), [], depth, line_labels) for depth in depths]
    
    def make_phase01_cos_circuits(self, depths, qid, line_labels):
        prep = gX01(qid) + gX01(qid) + gX12(qid) + gY01(qid) 
        meas = gY01_inv(qid) + gX12_inv(qid) + gX01(qid)*6
        return [make_rpe_circuit(gX01(qid), prep, meas, depth, line_labels) for depth in depths]

    def make_phase01_sin_circuits(self, depths, qid, line_labels):
        prep = gX01(qid) + gX01(qid) + gX12(qid) + gY01(qid) 
        meas = gY01_inv(qid) + gY12_inv(qid) + gX01(qid)*6
        return [make_rpe_circuit(gX01(qid), prep, meas, depth, line_labels) for depth in depths]

    


In [123]:
depths = [2**i for i in range(6)]
edesign = RPEDesign1QT(depths, qids[0], qids)
edesign.circuit_dict

{'Phase01': {'I': [Circuit(Gx01:Q0Gx01:Q0Gx12:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz12:Q0Gz12:Q0Gz12:Q0Gz12:Q0Gx12:Q0Gx12:Q0Gx12:Q0Gz01:Q0Gz01:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0@(Q0,Q1)),
   Circuit(Gx01:Q0Gx01:Q0Gx12:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz12:Q0Gz12:Q0Gz12:Q0Gz12:Q0Gx12:Q0Gx12:Q0Gx12:Q0Gz01:Q0Gz01:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0@(Q0,Q1)),
   Circuit(Gx01:Q0Gx01:Q0Gx12:Q0Gz01:Q0Gz01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gx01:Q0Gz01:Q0Gz01:

In [124]:
print(edesign.circ_list[0])

Qubit Q0 ---|Gx01|-|Gx01|-|Gx12|-|Gz01|-|Gz01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gz01|-|Gz01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gz01|-|Gz01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gz01|-|Gz01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gz12|-|Gz12|-|Gz12|-|Gz12|-|Gx12|-|Gx12|-|Gx12|-|Gz01|-|Gz01|-|Gz01|-|Gz01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|-|Gx01|---
Qubit Q1 ---|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |-|    |---



In [125]:
target_model.probabilities(edesign.circ_list[2])

OutcomeLabelDict([(('00',), 0.6249999999999961),
                  (('01',), -1.3955153506979294e-15),
                  (('02',), -7.571371115384644e-16),
                  (('10',), 0.2500000000000001),
                  (('11',), -6.806108575991225e-16),
                  (('12',), -5.489702944212475e-16),
                  (('20',), 0.1249999999999976),
                  (('21',), -1.8775700560231193e-15),
                  (('22',), -1.077673408879815e-15)])

In [127]:
ds = pygsti.data.simulate_data(target_model, edesign.circ_list, num_samples=1000)